In [53]:
import gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from sklearn.preprocessing import StandardScaler
# import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gpytorch
from scipy.linalg import solve
from gpytorch.kernels import RBFKernel,ScaleKernel,LinearKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from collections import namedtuple, deque


from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator

from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from IPython.display import display, clear_output

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [55]:

num_episodes = 2000
max_steps_per_episode = 200
test_episodes = 100
max_steps_per_test_episode=200
CI_parameter= 0.3
epsilon_start = 1.0
epsilon_end = 0.02
epsilon_decay_rate = 0.9985
gamma = 0.99
lr = 0.0025
buffer_size = 50000
buffer = deque(maxlen=buffer_size)
batch_size = 100
update_frequency = 10
nrow=20
ncol=20
sigma_noise=0.1
CI_start=0.4
CI_end=0.2
CI_decay_rate = 0.999
#  CI_parameter=0.5
# CI_parameter=0.5
penalty=60
#GP discretization bound
Position_lb=-4.8
Position_ub=4.8

Velocity_lb=-10000
Velocity_ub=10000
Pole_Angle_ld= -0.418 
Pole_Angle_rd= 0.418 



Pole_Angle_Velocity_lb= -10000
Pole_Angle_Velocity_ub=10000



Position_safe_lb=-1.9
Position_safe_ub=1.9
Pole_Angle_safe_ld= -0.15
Pole_Angle_safe_ud=   0.15



safety_bound=0.3

In [56]:
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size, seed, fc1_units=128, fc2_units=64):
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.to(device)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        return len(self.memory)
    

class DQNAgent:
    # Initialize the DQN agent
    def __init__(self, state_size, action_size, seed, lr):
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr)

        self.memory = ReplayBuffer(action_size, buffer_size=int(1e5), batch_size=64, seed=seed)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.add(state, action, reward, next_state, done)

        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            if len(self.memory) > 64:
                experiences = self.memory.sample()
                self.learn(experiences, gamma=0.99)

    # Choose an action based on the current state
    def act(self, state, eps=0.):
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
        
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state_tensor)
        self.qnetwork_local.train()

        if np.random.random() > eps:
            return action_values.argmax(dim=1).item()
        else:
            return np.random.randint(self.action_size)
        
    # Choose an safe action based on the current state and prediction
    def act_safe(self, state, model, CI_para, eps=0):
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
        
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state_tensor)
        self.qnetwork_local.train()
        
        possible_action=np.ones(self.action_size)
        for i in range(self.action_size):
            X=np.concatenate((state, np.array(i).reshape(1, )), axis=0)
            X_tensor=torch.from_numpy(X).float().unsqueeze(0).to(device)
            mean, log_var = model(X_tensor)
            ub= mean.item()+CI_para*np.sqrt(np.exp(log_var.item()))
            if ub>=safety_bound:
                possible_action[i]=0 
                # action_values[0][i]=-1
               
        if np.random.random() > eps:
            return action_values.argmax(dim=1).item()
        else:

            if np.sum(possible_action)==0:
                return action_values.argmax(dim=1).item()
            else:
                a=np.random.choice(self.action_size,size=1,p=possible_action/np.sum(possible_action))[0]
                return int(a)
            # return np.random.randint(self.action_size)
    # Learn from batch of experiences
    def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = zip(*experiences)
        states = torch.from_numpy(np.vstack(states)).float().to(device)
        actions = torch.from_numpy(np.vstack(actions)).long().to(device)
        rewards = torch.from_numpy(np.vstack(rewards)).float().to(device)
        next_states = torch.from_numpy(np.vstack(next_states)).float().to(device)
        dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(device)

        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        Q_expected = self.qnetwork_local(states).gather(1, actions)

        loss = F.mse_loss(Q_expected, Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.soft_update(self.qnetwork_local, self.qnetwork_target, tau=1e-3)

    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)
            
def cal_cost(state,action,next_state):
    cost1=0
    cost2=0
    if Position_safe_lb >=next_state[0]:
        cost1=Position_safe_lb-next_state[0]
    if Position_safe_ub <=next_state[0]:
        cost1=next_state[0]-Position_safe_ub
    if next_state[2]<=Pole_Angle_safe_ld:
        cost2=Pole_Angle_safe_ld-next_state[2]
    if next_state[2]>=Pole_Angle_safe_ud:
        cost2=next_state[2]-Pole_Angle_safe_ud
    return np.maximum(cost1,10*cost2)
       

class ProbabilisticRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.mean_head = nn.Linear(hidden_dim, 1)
        self.log_var_head = nn.Linear(hidden_dim, 1)  # log variance for stability

    def forward(self, x):
        h = self.shared(x)
        mean = self.mean_head(h)
        log_var = self.log_var_head(h)
        return mean, log_var  # we return log variance to keep it numerically stable
    
def gaussian_nll_loss(y_pred_mean, y_pred_log_var, y_true):
    # NLL for a Gaussian: log(sigma^2) + (y - mu)^2 / sigma^2
    loss = 0.5 * (y_pred_log_var + ((y_true - y_pred_mean) ** 2) / torch.exp(y_pred_log_var))
    return loss.mean()

In [57]:
def evaluate(new_agent,model):
    env = gym.make("CartPole-v1", render_mode="rgb_array")
    episode_rewards = []
    test_epi_length=np.zeros(test_episodes)
    test_cost_epi=np.zeros(test_episodes)
    test_vio_epi=np.zeros(test_episodes)
    for episode in range(test_episodes):
        state = env.reset()[0]
        episode_reward = 0
        done = False
        for step in range(max_steps_per_test_episode):
            action = new_agent.act_safe(state, model,CI_para=0.7,eps=0)
            # action = new_agent.act(state,eps=0)
            next_state, reward, done, truncated, _ = env.step(action)
            cost=cal_cost(state,action,next_state)
            episode_reward += reward
            state = next_state
            test_epi_length[episode]+=1
            test_cost_epi[episode]+=cost
            if cost>safety_bound:
                test_vio_epi[episode]+=1
            if done:
                break
            
        episode_rewards.append(episode_reward)

    average_reward = sum(episode_rewards) / test_episodes
    print(f"Average reward over {test_episodes} test episodes: {average_reward:.2f}")
    return average_reward,test_cost_epi,test_vio_epi,test_epi_length
    

In [ ]:
def one_rep():
    env = gym.make("CartPole-v1", render_mode="rgb_array")
    # Initialize the DQNAgent
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n
    new_agent = DQNAgent(input_dim, output_dim, seed=170715, lr = lr)
    #Initialize the Prediction NN
    model = ProbabilisticRegressor(input_dim=5, hidden_dim=128)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    iteration = np.zeros(num_episodes)
    violation=np.zeros(num_episodes)
    cost_episodes=np.zeros(num_episodes)
    for episode in range(num_episodes): #Gym versions >= 0.26 where env.reset() returns a tuple (obs, info)
        # Reset the environment
        state = env.reset()[0] #shape(4,)
        epsilon = max(epsilon_end, epsilon_start * (epsilon_decay_rate ** episode))
        # Run one episode
        for step in range(max_steps_per_episode):
            # # Choose and perform an action
            CI_parameter= CI_end+CI_start-max(CI_end, CI_start * (CI_decay_rate ** episode))
            # CI_parameter=0.2
            action = new_agent.act_safe(state, model,CI_parameter,epsilon)
            next_state, reward, done, truncated, _ = env.step(action)
            cost=cal_cost(state,action,next_state)
            buffer.append((state, action, reward-penalty*cost, next_state, done))
            cost_episodes[episode]+=cost
            if cost>safety_bound:
                violation[episode]+=1
            if len(buffer) >= batch_size:
                batch = random.sample(buffer, batch_size)
                new_agent.learn(batch, gamma) # Update the agent's knowledge
            
            #train BNN
            datapoints=np.concatenate((state, np.array(action).reshape(1, )), axis=0)
            data_tensor = torch.from_numpy(datapoints).float().unsqueeze(0).to(device)
            labels=np.array(cost).reshape(1,)
            labels_tensor=torch.from_numpy(labels).float().unsqueeze(0).to(device)
            optimizer.zero_grad()
            mean, log_var = model(data_tensor)
            loss = gaussian_nll_loss(mean, log_var, labels_tensor)
            loss.backward()
            optimizer.step()
            iteration[episode]+=1
            ub= mean.item()+0.1*np.sqrt(np.exp(log_var.item()))
            if ub>=safety_bound:
                break
            if violation[episode]>1.5:
                break
            state = next_state
            # Check if the episode has ended
            if done:
                break
    # Evaluate the agent's performance

    episode_rewards = []
    test_epi_length=np.zeros(test_episodes)
    test_cost_epi=np.zeros(test_episodes)
    test_vio_epi=np.zeros(test_episodes)
    for episode in range(test_episodes):
        state = env.reset()[0]
        episode_reward = 0
        done = False
        for step in range(max_steps_per_test_episode):
            action = new_agent.act_safe(state, model,CI_para=0.6,eps=0)
            # action = new_agent.act(state,eps=0)
            next_state, reward, done, truncated, _ = env.step(action)
            cost=cal_cost(state,action,next_state)
            episode_reward += reward
            state = next_state
            test_epi_length[episode]+=1
            test_cost_epi[episode]+=cost
            if cost>safety_bound:
                test_vio_epi[episode]+=1
            if done:
                break
            
        episode_rewards.append(episode_reward)

    average_reward = sum(episode_rewards) / test_episodes
    return average_reward,test_cost_epi,test_vio_epi,test_epi_length, new_agent,model,np.sum(violation)/np.sum(iteration)



In [59]:
episode=2200
CI_end+CI_start-max(CI_end, CI_start * (CI_decay_rate ** episode))

0.4000000000000001

In [60]:
num_multi_rep=10
mul_calls = [delayed(one_rep)() for rep in range(num_multi_rep)]
result_mul_delayed = Parallel(n_jobs=-3)(mul_calls)

In [61]:
num_multi_rep=10
agent_multi=[]
model_multi=[]
test_ave_reward_multi=np.zeros(num_multi_rep)
test_cost_epi_multi=np.zeros((num_multi_rep,test_episodes))
test_vio_epi_multi=np.zeros((num_multi_rep,test_episodes))
test_epi_length_multi=np.zeros((num_multi_rep,test_episodes))
train_viorate_epi_multi=np.zeros(num_multi_rep)
for i in range(num_multi_rep):
    test_ave_reward_multi[i]=np.copy(result_mul_delayed[i][0])
    test_cost_epi_multi[i]=np.copy(result_mul_delayed[i][1])
    test_vio_epi_multi[i]=np.copy(result_mul_delayed[i][2])
    test_epi_length_multi[i]=np.copy(result_mul_delayed[i][3])
    agent_multi.append(result_mul_delayed[i][4])
    model_multi.append(result_mul_delayed[i][5])
    train_viorate_epi_multi[i]=np.copy(result_mul_delayed[i][6])


In [62]:
vio_rate_multi=np.mean(test_vio_epi_multi/test_epi_length_multi,axis=1)
np.mean(test_ave_reward_multi),np.mean(vio_rate_multi),np.var(vio_rate_multi),vio_rate_multi

(199.853,
 0.0001323916009803908,
 1.5774782409135913e-07,
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00132392, 0.        , 0.        ]))

In [63]:
np.mean(train_viorate_epi_multi),np.std(train_viorate_epi_multi)/np.sqrt(10)

(0.011934400093376624, 0.0004035090363245207)

In [64]:


for i in range(num_multi_rep):
    test_ave_reward_multi[i],test_cost_epi_multi[i],test_vio_epi_multi[i],test_epi_length_multi[i]=evaluate(agent_multi[i],model_multi[i])


Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 198.95
Average reward over 100 test episodes: 200.00
Average reward over 100 test episodes: 200.00


In [66]:
vio_rate_multi=np.mean(test_vio_epi_multi/test_epi_length_multi,axis=1)
np.mean(test_ave_reward_multi),np.std(test_ave_reward_multi)/np.sqrt(10),np.mean(vio_rate_multi),np.var(vio_rate_multi),vio_rate_multi



(199.895,
 0.09961174629530502,
 0.00012988026247906861,
 1.518199432346858e-07,
 array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.0012988, 0.       , 0.       ]))